# TASK 1 - Find the prime numbers

Given a positive integer and a list of prime numbers, the objective is to find two prime numbers from the list that sum up to the given positive number. To accomplish this task, it's essential to consider the appropriate number of qubits and explain the validity of the proposal for all types of numbers.

There are multiple ways to approach this problem. One method involves systematically adding up various combinations of prime numbers from the list until the desired sum is achieved. To execute this approach, we would require three quantum registers: two to store the prime numbers being added together, and a third to store the final result. A quantum modulo addition operation can be employed to add the numbers in the first two registers. Subsequently, we measure the third register and perform a classical check to verify if it matches the desired output.

However, this approach has a drawback. Reinitializing the first two registers for each modulo addition operation incurs a time cost due to the classical-to-quantum overhead.

To address this issue, an alternative approach is to create a classical list of all possible combinations of prime numbers whose summation equals the given positive integer. Afterward, Grover's algorithm is utilized to verify whether the prime numbers required are present in this list. The number of qubits required will depend on the size of this list, which is determined by the number of primes you're considering.

## Imports

In [54]:
import numpy as np
import qiskit
from qiskit import *

import math

## Decomposition

In this section, we will create a function that takes as input a positive integer, and returns a list with combinations of all prime numbers that add to that integer.

In [167]:
# Function to check if a number is prime
def is_prime(num):
    if num <= 1:
        return False
    if num <= 3:
        return True
    if num % 2 == 0 or num % 3 == 0:
        return False
    i = 5
    while i * i <= num:
        if num % i == 0 or num % (i + 2) == 0:
            return False
        i += 6
    return True

# Function to decompose a number into two prime numbers
def decompose_into_two_primes(n):
    prime_combinations = []

    for i in range(2, n):
        j = n - i
        if is_prime(i) and is_prime(j):
            prime_combinations.append([i, j])

    return prime_combinations

# Example usage:
number_to_decompose = 8
combinations = decompose_into_two_primes(number_to_decompose)

# Print the prime combinations that sum up to the given number
print(f"Prime combinations that sum up to {number_to_decompose}:")
for combo in combinations:
    print(f"{combo[0]} + {combo[1]}")


Prime combinations that sum up to 8:
3 + 5
5 + 3


## Grover's algorithm

Grover's algorithm is a quantum algorithm that efficiently searches an unsorted database or performs an unstructured search, providing a quadratic speedup over classical algorithms. It's often used to find a marked item or solve certain types of computational problems faster than classical counterparts.

1. Construct an oracle that encodes the classical list of prime number combinations into a quantum superposition.

In [183]:
def calculate_amplitudes(integer_list):
    # Initialize an array of amplitudes with zeros
    max_value = max(integer_list)
    amplitudes = np.zeros(max_value + 1)

    # Set amplitudes to 1 for the integers in the list
    for i in integer_list:
        amplitudes[i] = 1

    # Normalize the amplitudes
    amplitudes /= np.linalg.norm(amplitudes)

    return amplitudes

# Example usage:
prime_list = [1, 3, 5, 7]
amplitudes = calculate_amplitudes(prime_list)
print(amplitudes)

[0.  0.5 0.  0.5 0.  0.5 0.  0.5]


In [169]:
def grover_oracle(marked_states):
    """Build a Grover oracle for multiple marked states

    Here we assume all input marked states have the same number of bits

    Parameters:
        marked_states (str or list): Marked states of oracle

    Returns:
        QuantumCircuit: Quantum circuit representing Grover oracle
    """
    if not isinstance(marked_states, list):
        marked_states = [marked_states]
    # Compute the number of qubits in circuit
    num_qubits = len(marked_states[0])

    qc = QuantumCircuit(num_qubits)
    # Mark each target state in the input list
    for target in marked_states:
        # Flip target bit-string to match Qiskit bit-ordering
        rev_target = target[::-1]
        # Find the indices of all the '0' elements in bit-string
        zero_inds = [ind for ind in range(num_qubits) if rev_target.startswith("0", ind)]
        # Add a multi-controlled Z-gate with pre- and post-applied X-gates (open-controls)
        # where the target bit-string has a '0' entry
        qc.x(zero_inds)
        qc.compose(MCMT(ZGate(), num_qubits - 1, 1), inplace=True)
        qc.x(zero_inds)
    return qc

2. Apply Grover iterations to amplify the amplitudes of the correct combinations while suppressing the incorrect ones. The number of Grover iterations required would be approximately proportional to the square root of the number of combinations.

In [170]:
def grover_search(marked_states, amplitudes):
    """
    Grover Search Algorithm

    This function uses Grover's algorithm to search for a marked state among possible states.

    Parameters:
    marked_states (list): List of states that are marked as the target states.
    amplitudes (list): Initial quantum state amplitudes.

    Returns:
    bool: True if the measured state with the highest probability is one of the marked states, False otherwise.
    """

    # Create the oracle for Grover's algorithm based on marked states
    oracle = grover_oracle(marked_states)

    # Create the Grover operator
    grover_op = GroverOperator(oracle)

    # Determine the optimal number of iterations for Grover's algorithm
    optimal_num_iterations = math.floor(
        math.pi / 4 * math.sqrt(2**grover_op.num_qubits / len(marked_states))
    )

    # Initialize a quantum circuit
    qc = QuantumCircuit(grover_op.num_qubits)

    # Initialize the quantum state using given amplitudes
    qc.initialize(amplitudes)

    # Apply Grover's operator the optimal number of times
    qc.compose(grover_op.power(optimal_num_iterations), inplace=True)

    # Measure all qubits in the quantum circuit
    qc.measure_all()

    # Use the Qiskit simulator to execute the quantum circuit
    simulator = Aer.get_backend('qasm_simulator')
    counts = execute(qc, backend=simulator).result().get_counts(qc)
    
    # Find the measured state with the highest probability
    max_state = max(counts, key=counts.get)
    
    # Check if the highest probability state is one of the marked states
    if max_state in marked_states:
        return True

    return False

3. Create the final function

In [181]:
def find_the_primes_numbers(number_1, prime_list):
    """"
    number_1 : integer value that is the positive number to decompose,
    number_2 : integer list that has two prime numbers to add to obtain number_1.
    Return the number_a and number_b
    """

     # use a framework that works with quantum circuits, qiskit, cirq, pennylane, etc. 

      # consider print your quantum circuit,

    combinations = decompose_into_two_primes(number_1)
    amplitudes = calculate_amplitudes(prime_list)
    
    for combination in combinations:
        bin1 = bin(combination[0])[2:]
        bin2 = bin(combination[1])[2:]
        max_length = max(len(bin1), len(bin2))

        # Pad the shorter binary string with leading zeros
        bin1 = bin1.zfill(max_length)
        bin2 = bin2.zfill(max_length)

        if grover_search(bin1, amplitudes) and grover_search(bin2, amplitudes):
            return int(bin1, 2), int(bin2, 2)
        else:
            continue

In [186]:
number_to_decompose = 8
prime_list = [1,3,5,7]
find_the_primes_numbers(number_to_decompose, prime_list)


(3, 5)

This is not working perfectly, I would need to work on the length of the list and number to decompose, but I dont have much time :/. Thank you for the opportunity!